###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de https://microdatos.dane.gov.co/index.php/catalog/643/get_microdata y corresponden a datos cuya fuente primaria es el Departamento Administrativo Nacional de Estadisticas - DANE

In [1]:
import os
from pathlib import Path
import pandas as pd
import zipfile
import urllib.request

### Descarga de set de datos y carga como df

Contiene el resultado de la descarga de 33 conjuntos de datos, uno por cada departamento más Bogotá, que contiene los microdatos de los hogares del territorio nacional del  censo de población y vivienda realizado en 2018.

El diccionario de datos de estos dataset puede consultarse en: https://microdatos.dane.gov.co/index.php/catalog/643/data-dictionary/F9?file_name=HOGARES

In [2]:
# Funcion para descargar, extraer y cargar datos del censo 2018
def load_census_data(department_name, url, file_prefix):
    """
    Descarga y extrae datos del Censo 2018 de un departamento desde una URL.

    Descarga un archivo .zip, extrae el subarchivo "_CSV", y carga el CSV que coincide con `file_prefix`.

    Args:
        department_name (str): Nombre del departamento.
        url (str): URL de descarga del archivo .zip.
        file_prefix (str): Prefijo del CSV a buscar (viviendas, hogares, personas).

    Returns:
        pd.DataFrame or None: DataFrame con los datos del CSV o None si no se encuentra.

    El nombre del archivo CSV se forma con:
        f"{file_prefix}_{department_name[-2:]}.CSV"
    """

    # Ruta y URL del archivo zip a descargar
    zip_path = Path(f"datasets/{department_name}.zip")

    # Verificar si el archivo .zip ya existe, si no, descargarlo
    if not zip_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        urllib.request.urlretrieve(url, zip_path)

    # Extraer el archivo .zip descargado
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(path="datasets")

    # Identificar dinámicamente la carpeta extraída
    extracted_folder = None
    for folder in Path("datasets").iterdir():
        if folder.is_dir() and department_name.split('_')[0] in folder.stem:
            extracted_folder = folder
            break

    if not extracted_folder:
        print(f"No se pudo encontrar la carpeta extraída para {department_name}")
        return None

        # Identificar el archivo .zip con sufijo "_CSV" y extraer su contenido
    csv_zip_path = None
    for item in extracted_folder.iterdir():
        if item.is_file() and item.suffix == '.zip' and "_CSV" in item.stem:
            csv_zip_path = item
            break

    # Si se encuentra el archivo _CSV, lo descomprimimos
    if csv_zip_path:
        with zipfile.ZipFile(csv_zip_path, 'r') as csv_zip_ref:
            csv_zip_ref.extractall(path=extracted_folder / "CSV")

    # Definir la ruta donde se descomprimieron los archivos CSV
    csv_folder = extracted_folder / "CSV"

    # Cargar el archivo CSV específico para el tipo de información solicitado
    file_to_load = f"{file_prefix}_{department_name[:2]}.CSV"
    csv_file_path = csv_folder / file_to_load
    if csv_file_path.is_file():
        return pd.read_csv(csv_file_path)
    else:
        print(f"Archivo no encontrado para {department_name}")
        return None


In [3]:
# Función para procesar una lista de URLs y concatenar los archivos
def process_multiple_departments(urls, file_prefix):
    """
    Procesa múltiples departamentos y concatena los datos en un único DataFrame.

    Itera sobre una lista de URLs y departamentos, usando `load_census_data` para cargar los CSVs
    y los concatena en un solo DataFrame.

    Args:
        urls (list of tuple): Lista de tuplas (nombre del departamento, URL).
        file_prefix (str): Prefijo del CSV a buscar (viviendas, hogares, personas).

    Returns:
        pd.DataFrame: DataFrame con los datos combinados de todos los departamentos.
    """

    all_data = []
    for department_name, url in urls:
        df = load_census_data(department_name, url, file_prefix)
        if df is not None:
            all_data.append(df)

    # Concatenar todos los DataFrames en uno solo
    return pd.concat(all_data, ignore_index=True)


In [4]:
# Lista de departamentos y URLs
departments_urls = [
     ("05_Antioquia", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12143"),
     ("08_Atlantico", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12144"),
     ("11_Bogota", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12145"),
     ("13_Bolivar", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12146"),
     ("15_Boyaca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12147"),
     ("17_Caldas", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12148"),
     ("18_Caqueta", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12149"),
     ("19_Cauca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12150"),
     ("20_Cesar", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12151"),
     ("23_Cordoba", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12152"),
     ("25_Cundinamarca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12153"),
     ("27_Choco", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12154"),
     ("41_Huila", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12155"),
     ("44_La Guajira", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12156"),
     ("47_Magdalena", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12157"),
     ("50_Meta", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12158"),
     ("52_Narino", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12159"),
     ("54_Norte de Santander", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12160"),
     ("63_Quindio", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12161"),
     ("66_Risaralda", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12162"),
     ("68_Santander", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12163"),
     ("70_Sucre", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12164"),
     ("73_Tolima", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12165"),
     ("76_Valle del Cauca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12166"),
     ("81_Arauca", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12167"),
     ("85_Casanare", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12168"),
     ("86_Putumayo", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12169"),
     ("88_San Andres Providencia y SantaCatalina", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12170"),
     ("91_Amazonas", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12171"),
     ("94_Guainia", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12172"),
     ("95_Guaviare", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12173"),
     ("97_Vaupes", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12174"),
     ("99_Vichada", "https://microdatos.dane.gov.co/index.php/catalog/643/download/12175"),
 ]

# Procesar la información de viviendas (file_prefix es "CNPV2018_2HOG")
censo_hogares_2018_dane = process_multiple_departments(departments_urls, "CNPV2018_2HOG_A2")


- Tamaño del dataset obtenido

In [5]:
censo_hogares_2018_dane.shape

(14252829, 13)

- Estructura del dataset creado a partir de las descargas

In [6]:
censo_hogares_2018_dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14252829 entries, 0 to 14252828
Data columns (total 13 columns):
 #   Column            Dtype  
---  ------            -----  
 0   TIPO_REG          int64  
 1   U_DPTO            int64  
 2   U_MPIO            int64  
 3   UA_CLASE          int64  
 4   COD_ENCUESTAS     int64  
 5   U_VIVIENDA        int64  
 6   H_NROHOG          float64
 7   H_NRO_CUARTOS     float64
 8   H_NRO_DORMIT      float64
 9   H_DONDE_PREPALIM  float64
 10  H_AGUA_COCIN      float64
 11  HA_NRO_FALL       float64
 12  HA_TOT_PER        float64
dtypes: float64(7), int64(6)
memory usage: 1.4 GB


- Analisis de las columnas para determinar su importancia para el proyecto

In [7]:
columns = censo_hogares_2018_dane.columns

In [8]:
# Significado de columna según el diccionario de datos de los dataset
meaning_columns= {'TIPO_REG': 'Tipo de registro', 'U_DPTO': 'Departamento', 'U_MPIO': 'Municipio',
                  'UA_CLASE': 'Clase', 'COD_ENCUESTAS': 'Codigo encuenta', 'U_VIVIENDA': 'Numero de orden en la vivienda',
                  'H_NROHOG': 'Numero de hogar en la vivienda', 'H_NRO_CUARTOS': 'Numero total de cuartos',
                  'H_NRO_DORMIT': 'Numero de cuartos para dormir', 'H_DONDE_PREPALIM': 'Lugar donde preparan los alimentos',
                  'H_AGUA_COCIN': 'Fuentes de agua para preparar los alimentos', 'HA_NRO_FALL': 'Total fallecidos en ekl hogar',
                  'HA_TOT_PER': 'Total personas en el hogar'}

In [9]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in columns:
  print(f'{col}: {censo_hogares_2018_dane[col].nunique()}')

TIPO_REG: 1
U_DPTO: 33
U_MPIO: 581
UA_CLASE: 3
COD_ENCUESTAS: 13490169
U_VIVIENDA: 1761
H_NROHOG: 35
H_NRO_CUARTOS: 21
H_NRO_DORMIT: 21
H_DONDE_PREPALIM: 7
H_AGUA_COCIN: 12
HA_NRO_FALL: 14
HA_TOT_PER: 36


In [10]:
# Mostrar los valores únicos de cada columna que tenga menos de 50 categorías
for col in columns:
  l = censo_hogares_2018_dane[col].nunique()
  if l < 50:
    print(f'{col}: {censo_hogares_2018_dane[col].unique()}')

TIPO_REG: [2]
U_DPTO: [ 5  8 11 13 15 17 18 19 20 23 25 27 41 44 47 50 52 54 63 66 68 70 73 76
 81 85 86 88 91 94 95 97 99]
UA_CLASE: [1 2 3]
H_NROHOG: [ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 nan 35. 34. 33. 31. 30. 29. 28. 27. 26. 25. 23. 22. 21. 24. 20. 19. 32.]
H_NRO_CUARTOS: [ 4.  3.  2.  1.  5.  7.  6.  8.  9. 12. 99. 10. 13. 16. 17. 11. 15. 14.
 20. 18. 19. nan]
H_NRO_DORMIT: [ 2.  3.  1.  4.  6.  5.  7.  8. 99.  9. 14. 10. 16. 12. 11. 17. 20. 18.
 15. 13. 19. nan]
H_DONDE_PREPALIM: [ 1.  4.  6.  2.  3.  5.  9. nan]
H_AGUA_COCIN: [ 1. nan  3.  2.  8. 99.  6. 11.  4.  5.  7.  9. 10.]
HA_NRO_FALL: [nan  1.  2.  3.  5.  4.  8.  9.  6. 10.  7. 12. 15. 18. 13.]
HA_TOT_PER: [ 5.  3.  4.  2.  1.  6.  7.  9.  8. 10. 11. 12. 13. 16. 15. 17. 19. 14.
 26. 21. 18. 20. 27. nan 28. 24. 25. 22. 31. 30. 23. 39. 29. 38. 32. 40.
 36.]


In [11]:
# Eliminar columnas que se considera innecesarias para el proyecto
important_columns = ['U_DPTO', 'U_MPIO', 'UA_CLASE', 'H_NROHOG', 'H_NRO_CUARTOS','H_NRO_DORMIT', 'H_DONDE_PREPALIM', 'H_AGUA_COCIN', 'HA_TOT_PER']
censo_hogares_2018_dane = censo_hogares_2018_dane[important_columns]
censo_hogares_2018_dane.head()

,U_DPTO,U_MPIO,UA_CLASE,H_NROHOG,H_NRO_CUARTOS,H_NRO_DORMIT,H_DONDE_PREPALIM,H_AGUA_COCIN,HA_TOT_PER
0,5,1,1,1.0,4.0,2.0,1.0,1.0,5.0
1,5,1,1,1.0,4.0,3.0,1.0,1.0,3.0
2,5,1,1,1.0,3.0,2.0,1.0,1.0,3.0
3,5,1,1,2.0,3.0,2.0,1.0,1.0,4.0
4,5,1,1,1.0,3.0,2.0,1.0,1.0,3.0


### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.

En el presente df los códigos de los municipios y los departamentos están almacenados por aparte y en formato int64


In [12]:
# valores codigo departamento
print(f'Valores unicos codigos depto: {censo_hogares_2018_dane["U_DPTO"].unique()}')
print(f'Valores unicos codigos mpios: {censo_hogares_2018_dane["U_MPIO"].unique()}')

Valores unicos codigos depto: [ 5  8 11 13 15 17 18 19 20 23 25 27 41 44 47 50 52 54 63 66 68 70 73 76
 81 85 86 88 91 94 95 97 99]
Valores unicos codigos mpios: [  1   2   4  21  30  31  34  36  38  40  42  44  45  51  55  59  79  86
  88  91  93 101 107 113 120 125 129 134 138 142 145 147 148 150 154 172
 190 197 206 209 212 234 237 240 250 264 266 282 284 306 308 310 313 315
 318 321 347 353 360 361 364 368 376 380 390 400 411 425 440 467 475 480
 483 490 495 501 541 543 576 579 585 591 604 607 615 628 631 642 647 649
 652 656 658 659 660 664 665 667 670 674 679 686 690 697 736 756 761 789
 790 792 809 819 837 842 847 854 856 858 861 873 885 887 890 893 895  78
 137 141 296 372 421 433 436 520 549 558 560 573 606 634 638 675 685 758
 770 832 849   6  52  62  74 140 160 188 222 244 248 268 300 430 442 458
 468 473 580 600 620 650 654 655 657 673 683 688 744 760 780 810 836 838
 894  22  47  87  90  92  97 104 106 109 114 131 135 162 176 180 183 185
 187 189 204 215 218 223 224 226 23

- Longitud de los códigos de departamentos y municipios en el df

In [13]:
# Convertir a strings
censo_hogares_2018_dane[['U_DPTO','U_MPIO']] = censo_hogares_2018_dane[['U_DPTO','U_MPIO']].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_depto = censo_hogares_2018_dane['U_DPTO'].apply(len)
longitud_mpio = censo_hogares_2018_dane['U_MPIO'].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los dptos tiene longitudes de :{longitud_depto.unique()} caracteres')
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los dptos tiene longitudes de :[1 2] caracteres
Los códigos de los mpios tiene longitudes de :[1 2 3] caracteres


- Convertir los códigos de los departamentos en un string de 2 digitos y los de los municipios en un string de 3 digitos para que coincidan con el formato del código almacenado en la base de datos

Rellenamos con ceros a la izquierda aquellos códigos del departamento que solo tienen un dígito y aquellos códigos de los municipios que solo tienen 1 y 2 dígitos


In [14]:
# Función para formatear los códigos
def formatear_codigo(codigo, longitud):
    return codigo.zfill(longitud)

# Aplicar la función a las columnas y crear la nueva columna
censo_hogares_2018_dane['U_DPTO'] = censo_hogares_2018_dane['U_DPTO'].apply(formatear_codigo, args=(2,))
censo_hogares_2018_dane['U_MPIO'] = censo_hogares_2018_dane['U_MPIO'].apply(formatear_codigo, args=(3,))

- Crear nueva columna que contenga el codigo del departamento mas el codigo del municipio

In [15]:
# Nueva columna de código de municipio con longitud de 5 caracteres
censo_hogares_2018_dane['COD_DEP_MPIO'] = censo_hogares_2018_dane['U_DPTO'] +censo_hogares_2018_dane['U_MPIO']

# Verificar
censo_hogares_2018_dane['COD_DEP_MPIO']

,COD_DEP_MPIO
0,05001
1,05001
2,05001
3,05001
4,05001
...,...
14252824,99773
14252825,99773
14252826,99773
14252827,99773


In [16]:
# Eliminar columnas 'U_DPTO' y 'U_MPIO' que ya no son necesarias
censo_hogares_2018_dane = censo_hogares_2018_dane.drop(['U_DPTO', 'U_MPIO'], axis=1)
censo_hogares_2018_dane.head(3)

,UA_CLASE,H_NROHOG,H_NRO_CUARTOS,H_NRO_DORMIT,H_DONDE_PREPALIM,H_AGUA_COCIN,HA_TOT_PER,COD_DEP_MPIO
0,1,1.0,4.0,2.0,1.0,1.0,5.0,05001
1,1,1.0,4.0,3.0,1.0,1.0,3.0,05001
2,1,1.0,3.0,2.0,1.0,1.0,3.0,05001


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [17]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


In [18]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [19]:
# Comparar listas de códigos
compare_lists(censo_hogares_2018_dane['COD_DEP_MPIO'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en victims_by_type_of_crime_UnidadVictimas", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en victims_by_type_of_crime_UnidadVictimas que no están en Códigos de municipios dept_mpios_codes:
{'94663'}


Nota: El codigo 94663 pertence a Mapiripana una poblacion del Guania que en la tabla geolocalizada de municipios de la bd figura como corregimiento del municipio Barranco Minas de Guania. Por lo tanto reemplazamos este codigo por el de Municipio al que pertenece

In [20]:
censo_hogares_2018_dane['COD_DEP_MPIO'] = censo_hogares_2018_dane['COD_DEP_MPIO'].replace( '94663', '94343')

- Volvemos a veridicar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [21]:
# Comparar listas de códigos
compare_lists(censo_hogares_2018_dane['COD_DEP_MPIO'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en victims_by_type_of_crime_UnidadVictimas", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en victims_by_type_of_crime_UnidadVictimas que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [22]:
# Adicionar columna para trazabilidad de la fuente
censo_hogares_2018_dane['SOURCE_ID'] = 54

In [23]:
censo_hogares_2018_dane.columns

Index(['UA_CLASE', 'H_NROHOG', 'H_NRO_CUARTOS', 'H_NRO_DORMIT',
       'H_DONDE_PREPALIM', 'H_AGUA_COCIN', 'HA_TOT_PER', 'COD_DEP_MPIO',
       'SOURCE_ID'],
      dtype='object')

In [ ]:
#Estructura final del dataset a integrar a la base de datos
censo_hogares_2018_dane.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14252829 entries, 0 to 14252828
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   UA_CLASE          int64  
 1   H_NROHOG          float64
 2   H_NRO_CUARTOS     float64
 3   H_NRO_DORMIT      float64
 4   H_DONDE_PREPALIM  float64
 5   H_AGUA_COCIN      float64
 6   HA_TOT_PER        float64
 7   COD_DEP_MPIO      object 
 8   SOURCE_ID         int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 978.7+ MB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
censo_hogares_2018_dane.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/censo_hogares_2018_dane.csv', index=False)